In [1]:
#for debuging on my computer
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
from args_parameter import args
from PrepareData import ACCESS_BARRA_v1,ACCESS_BARRA_v2

import torch,os,torchvision
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim

# from PIL import Image
import time
from sklearn.model_selection import StratifiedShuffleSplit
import model
import utility
from tqdm import tqdm
import math
import xarray as xr
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr,compare_mse

args.file_ACCESS_dir="E:/climate/access-s1/"
# args.file_ACCESS_dir="H:/climate/access-s1/"

args.file_BARRA_dir="C:/Users/JIA059/barra/"
# args.file_BARRA_dir="D:/dataset/accum_prcp/"

args.n_threads=4
args.channels=1
args.batch_size=4

# ensemble=['e01','e02']
args.ensemble=2
access_rgb_mean= 2.9067910245780248e-05*86400

leading_time=217
args.leading_time_we_use=7
# args.lr=0.001

init_date=date(1970, 1, 1)
start_date=date(1990, 1, 2)
end_date=date(1990,12,25) #if 929 is true we should substract 1 day
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]
print(access_rgb_mean)

print("training statistics:")
print("  ------------------------------")
print("  trainning name  |  %s"%args.train_name)
print("  ------------------------------")
print("  num of channels | %5d"%args.channels)
print("  ------------------------------")
print("  num of threads  | %5d"%args.n_threads)
print("  ------------------------------")
print("  batch_size      | %5d"%args.batch_size)


2.5114674452354135
training statistics:
  ------------------------------
  trainning name  |  temp01
  ------------------------------
  num of channels |     1
  ------------------------------
  num of threads  |     4
  ------------------------------
  batch_size      |     4


In [2]:

train_transforms = transforms.Compose([
#     transforms.Resize(IMG_SIZE),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
    transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
])

data_set=ACCESS_BARRA_v2(start_date,end_date,transform=train_transforms,args=args)
train_data,val_data=random_split(data_set,[int(len(data_set)*0.8),len(data_set)-int(len(data_set)*0.8)])


print("Dataset statistics:")
print("  ------------------------------")
print("  total | %5d"%len(data_set))
print("  ------------------------------")
print("  train | %5d"%len(train_data))
print("  ------------------------------")
print("  val   | %5d"%len(val_data))
data_set[0]


=> BARRA_R & ACCESS_S1 loading
=> from 1990/01/02 to 1990/12/25
C:/Users/JIA059/barra/
no file or no permission!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Dataset statistics:
  ------------------------------
  total |   603
  ------------------------------
  train |   482
  ------------------------------
  val   |   121
end loading one data,time cost 2.912232


(tensor([[[ 1.2529,  1.2445,  1.2274,  ...,  0.0319,  0.0267,  0.0236],
          [ 1.2645,  1.2333,  1.1783,  ...,  0.0351,  0.0306,  0.0283],
          [ 1.2370,  1.1397,  0.9835,  ...,  0.0359,  0.0280,  0.0235],
          ...,
          [ 7.2038,  5.9898,  4.1904,  ...,  0.1658,  0.1438,  0.1397],
          [ 9.0561,  8.4482,  7.5293,  ...,  0.2855,  0.2619,  0.2579],
          [ 9.0242,  9.9771, 11.2926,  ...,  0.4526,  0.5105,  0.5530]]],
        dtype=torch.float64),
 tensor([[[ 0.4657,  0.5828,  0.7021,  ...,  0.0827,  0.0860,  0.1284],
          [ 0.5201,  0.6066,  0.6744,  ...,  0.1019,  0.0852,  0.0895],
          [ 0.5973,  0.6674,  0.6914,  ...,  0.0830,  0.0894,  0.0736],
          ...,
          [10.1682, 10.1977,  8.8950,  ...,  1.5114,  1.7257,  1.7105],
          [ 9.2418,  9.7900, 10.1753,  ...,  1.6090,  1.3181,  1.5545],
          [ 6.9266,  8.3618,  7.6947,  ...,  1.2686,  1.2783,  1.2558]]],
        dtype=torch.float64),
 tensor(19900110),
 tensor(1))

In [3]:
##set a the dataLoader
train_dataloders =DataLoader(train_data,
                                        batch_size=args.batch_size,
                                        shuffle=False,
                            num_workers=args.n_threads)
val_dataloders =DataLoader(val_data,
                                        batch_size=args.batch_size,
                                        shuffle=False,
                          num_workers=args.n_threads)
##

args.cpu=True
def prepare( l, volatile=False):
    device = torch.device('cpu' if args.cpu else 'cuda')
    def _prepare(tensor):
        if args.precision == 'half': tensor = tensor.half()
        return tensor.to(device)

    return [_prepare(_l) for _l in l]

checkpoint = utility.checkpoint(args)
net = model.Model(args, checkpoint).double()
args.lr=0.001
criterion = nn.L1Loss()
optimizer_my = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer_my, step_size=7, gamma=0.1)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_my, gamma=0.9)
# torch.optim.lr_scheduler.MultiStepLR(optimizer_my, milestones=[20,80], gamma=0.1)


Making model...
accesss-s1 mean (0.4690)


In [4]:

#training
max_error=np.inf
# train_loss=
# print("batch size %d"%args.batch_size)
# print("batch size %d"%args.batch_size)

for e in range(args.epochs):
    #train
    print("start Training for %d"%e)
#     net.train()
    loss=0
    start=time.time()
    print("start Training for start loading data")

    for batch, (lr, hr,_,_) in enumerate(train_dataloders):
        print("start first batch train")

        lr, hr = prepare([lr, hr])
#         optimizer_my.zero_grad()
#         with torch.set_grad_enabled(True):
#             sr = net(lr, 0)
# #         error = criterion(sr[:,:,:,0:403], hr)
#             running_loss =criterion(sr, hr)
#             loss+=running_loss 
#         running_loss.backward()
#         optimizer_my.step()
        print("end first batch train")

        
    #validation
    net.eval()
    start=time.time()
    with torch.no_grad():
        eval_psnr=0
        eval_ssim=0
#         tqdm_val = tqdm(val_dataloders, ncols=80)
        for idx_img, (lr, hr,date,_) in enumerate(val_dataloders):
            print("start first batch validation")

            lr, hr = prepare([lr, hr])
#             sr = net(lr, 0)
#             val_loss=criterion(sr, hr)
#             for ssr,hhr in zip(sr,hr):
#                 eval_psnr+=compare_psnr(ssr[0].cpu().numpy(),hhr[0].cpu().numpy(),data_range=(hhr[0].cpu().max()-hhr[0].cpu().min()).item() )
#                 eval_ssim+=compare_ssim(ssr[0].cpu().numpy(),hhr[0].cpu().numpy(),data_range=(hhr[0].cpu().max()-hhr[0].cpu().min()).item() ) 
            print("end first batch validation")

    print("epoche: %d,time cost %f s, lr: %f, train_loss: %f,validation loss:%f "%(
              e,
              time.time()-start,
              optimizer_my.state_dict()['param_groups'][0]['lr'],
              running_loss.item()/len(train_data),
              val_loss
         ))
        
    if running_loss<max_error:
        print("saving")
        max_error=running_loss
        if not os.path.exists("./model/save/"+args.train_name+"/"):
            os.mkdir("./model/save/"+args.train_name+"/")
#         torch.save(net,"./model/save/"+args.train_name+"/"+str(e)+".pkl")
        print("end Training for %d"%e)
        

start Training for 0
start Training for start loading data
start first batch train
end first batch train
start first batch train
end first batch train
start first batch train
end first batch train
start first batch train
end first batch train


KeyboardInterrupt: 